# Set up the environment
1. Install the necessary libraries
2. Authenticate with GEE

In [ ]:
!pip install geopandas
!pip install rasterio
!pip install fulcrum
!pip install rasterstats
!pip install StringIO
#!pip install ee
!pip install earthengine-api --upgrade
!pip install importlib
!pip install open_clip_torch
!pip install contextily

In [ ]:
import io
from io import StringIO
import contextily as ctx
import shapely.geometry
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import json
import reportlab
from reportlab.lib.pagesizes import A4
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import math
import warnings
from datetime import datetime, timedelta
import ee # import Google earth engine module
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from fulcrum import Fulcrum
from rasterio.transform import from_origin
from rasterstats import zonal_stats

#Authenticate the Google Earth engine with Google account
# First setup only, no need to run this after first run
#ee.Authenticate()
#ee.Initialize()
# warnings.filterwarnings('ignore')

# Get Fulcrum Data

In [ ]:
with open("/credentials.json") as c:
	credentials = json.load(c)

fulcrum = Fulcrum(key=credentials['fulcrum_api'])

In [ ]:
# todo: add project name and use throughout the file
projectName = 'KKR'

In [ ]:
formdata = fulcrum.forms.find(credentials['TMO_TI_form'])

recordCount = formdata['form']['record_count']
pages = math.ceil(recordCount / 5000)
# way cleaner than my script. Thanks!
data = []
for p in range(1, pages + 1):
  dataPage = fulcrum.records.search(
    url_params={'form_id': credentials['TMO_TI_form'], 'page': p, 'per_page': 5000})['records']
  data.extend(dataPage)

In [ ]:
# simplify data and un-nest values from json. Add all trees without spread get a 10m value.
for record in data:
	if 'f6ef' in record['form_values']:
		record['health'] = ''.join(record['form_values']['f6ef']['choice_values'])
	else:
		record['health'] = 'NaN'

	if '0a3e' in record['form_values']:
		record['structure'] = ''.join(record['form_values']['0a3e']['choice_values'])
	else:
		record['structure'] = 'NaN'

	if '009b' in record['form_values']:
		record['height'] = ''.join(record['form_values']['009b'])
	else:
		record['height'] = 'NaN'

	if '7a77' in record['form_values']:
		record['DBH'] = ''.join(record['form_values']['7a77'])
	else:
		record['DBH'] = 'NaN'

	if '6e5e' in record['form_values']:
		record['source'] = ''.join(record['form_values']['6e5e']['choice_values'])
	else:
		record['source'] = 'NaN'

	if 'b96d' in record['form_values']:
		record['spread'] = ''.join(record['form_values']['b96d'])
	else:
		record['spread'] = '7'

	if '3fee' in record['form_values']:
		record['species'] = ''.join(record['form_values']['3fee']['choice_values'])
	else:
		record['species'] = 'No Name - لا يوجد اسم'

	record['genus']= record['species'].split(' ')[0]

	if record['genus'] in ['no', '', 'No']:
		record['genus'] = 'No name'

	# 	select dead and missing from inventory and lable them future trees
	if record['status'] in ['Missing tree - لا يوجد شجرة', 'Dead tree - شجرة ميتة', 'Plant tree - ازرع شجرة']:
		record['existing'] = 'no'
	else:
		record['existing'] = 'yes'

	if record['genus'] in ['Phoenix', 'Washingtonia']:
		record['Class'] = 'Palm'
	elif record['genus'] == 'No name':
		record['Class'] = 'Not identified'
	else:
		record['Class'] = 'Shade'

	if record['status'] == 'No action required - لا حاجة لأي إجراء':
		record['status'] = 'No action required'
	elif record['status'] == 'Request of activation - طلب تفعيل':
		record['status'] = 'Request of activation'
	elif record['status'] == 'Request of inspection (RFI) - طلب معاينة':
		record['status'] = 'Request of inspection'
	elif record['status'] == 'Escalation issue - قضية تصعيد':
		record['status'] = 'Escalation issue'
	else:
		record['status'] = 'Issue in place'

'''
if record['genus'] in ['Phoenix', 'Washingtonia', 'Acacia', '', '','']:
		record['Type'] = 'Tree'
	else:
		record['Type'] = 'Shrubs'
'''

In [ ]:
# get unique project IDs in a list
res = list()
for record in data:
    project_id = record['project_id']
    if project_id not in res:
            res.append(project_id)


# create dict with matching names
result = {}
for i in res:
    try:
        project_info = fulcrum.projects.find(i)['project']
        result[i] = project_info['name']
    except:
        result[i] = 'Unknown Project'


# add project names to org data
for record in data:
    record['project'] = result.get(record['project_id'], 'Unknown')

In [ ]:
df = pd.DataFrame(data)
df.drop(columns=['id', 'form_id', 'created_by_id', 'version','id', 'created_at','updated_at',
                 'created_by', 'assigned_to_id', 'altitude', 'speed', 'course', 'horizontal_accuracy',
                 'vertical_accuracy', 'updated_by', 'updated_by_id', 'created_location', 'updated_location',
                 'created_duration', 'updated_duration', 'edited_duration', 'project_id', 'record_series_id', 'assigned_to', 'form_values'], axis=1, inplace=True)

inventory = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.longitude, df.latitude))
inventory.crs = 'EPSG:4326'
inventory = inventory.to_crs(32638)
# inventory_3 = inventory.copy()

In [ ]:
inventory = inventory.drop_duplicates(['geometry'])

In [ ]:
print(len(inventory))

# Define related functions

1- read file function

2- plot map function

In [ ]:
# define read_data function to load geojson files and project them
def read_data(URL, crs):
  # URL= "r'"+URL+"'"
  data = gpd.read_file(URL)
  data = data.to_crs(crs)

  return data

In [ ]:
def plot_map(dataset, boundary, column, title, savefig):
    # todo: you define a plot function here but do not use it at all
    # Reproject data to Web Mercator projection (EPSG:3857)
    dataset = dataset.to_crs(epsg=3857)

    # Plot the GeoDataFrame with colors and labels.#cmap: Set3, plasma, magma, inferno, viridis, coolwarm
    fig, ax = plt.subplots(figsize=(15, 15))
    dataset.plot(column=column, cmap='coolwarm', ax=ax, legend=True, markersize=2)
    boundary.to_crs(epsg=3857).plot(edgecolor="black", facecolor="none", ax=ax)

    # Add basemap
    source = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
    ctx.add_basemap(ax, source=source)
    ax.set_axis_off()

    # add a title
    plt.title(title, size=18)

    # Save the plot to a PNG file
    plt.savefig(savefig, dpi='figure', bbox_inches='tight')

    # Show the plot
    plt.show()

# Get other Data:

1.   KKR and KSR boundaries
2.   Riyadh boundary

In [ ]:
#Call read_data function to load data
# 1- KKR Boundaries
KKRboundarygdf = read_data(r"/Users/philipp/Projects/PycharmProjects/fulcrum-python-tools/review/KKR.gpkg", 32638)
# 2- Riyadh boundary
Riyadhboundarygdf = read_data(r"/Users/philipp/BPLA Dropbox/03 Planning/BAHGA-Greening-Arriyadh/03_Drawings/GIS-data/RUH-urban-environment/urban-environs.shp", 32638)
##PR: i added the the r in front of the path as some of our paths have spaces and other unescaped characters.  

In [ ]:
# KKRboundarygdf.plot()
# Riyadhboundarygdf.plot()

# Filter data


1.   Remove points outside Riyadh City boundary
2.   Check overlay with KKR and KSR boundaries


In [ ]:
# Check for intersection
intersectionriyadh = inventory.geometry.within(Riyadhboundarygdf.geometry.iloc[0])

# Create a new column 'rcrc' in the Fulcrum data and set it to 'yes'
inventory['withinRiyadh'] = 'no'

# Set the 'overlayRiyadh' column to 'yes' for rows where there is an intersection
inventory.loc[intersectionriyadh, 'withinRiyadh'] = 'yes'

print(inventory["withinRiyadh"].value_counts())

In [ ]:
inventory_1 = inventory[inventory['withinRiyadh'] == 'yes']
print(inventory_1.shape)

In [ ]:
#	Filter inventory by KKRKSRboundarygdf
inventory_2 = gpd.sjoin(inventory_1, KKRboundarygdf, how = "left")
inventory_2['points'] = np.where(np.isnan(inventory_2['index_right']), 0, 1)
del (inventory_2['index_right'])

inventory_2.loc[inventory_2['points'] == 0, 'overlayK'] = 'outside KKR/KSR'
inventory_2['overlayK'] = inventory_2['overlayK'].fillna('inside KKR/KSR')

print(inventory_2['overlayK'].value_counts())

# inventory_2.drop(columns=['altitudeMo','tessellate', 'extrude', 'visibility', 'drawOrder', 'icon', 'points', 'Name', 'timestamp', 'begin', 'end', 'fid', 'descriptio', 'English_na'], axis=1, inplace=True)

In [ ]:
testKKR = inventory[inventory['project']=='KKR']
testKKR.shape

# Output maps

1- Masterplan Current Status

Input:

        Fulcrum ([TMO] combined tree inventory)

Output:

        Total trees:
              trees inside KKR and KSR
              trees outside KKR and KSR

page 12

In [ ]:
inventory = inventory.to_crs(epsg=3857)

color_dict = dict.fromkeys(inventory['project'].unique(),"fuchsia")
color_dict['KKR'] = 'green'
# todo: this needs to be dynamic as the data will always vary between projects and different points in time. 

# Map the color_dict values to the inventory["project"] to be correctly mapped on the map
inventory["color"] = inventory["project"].map(color_dict)

# plot data and boundary based on the specified inventory['color']
fig, ax = plt.subplots(figsize=(12, 12))
inventory.plot(ax=ax, color=inventory["color"], legend=True, markersize=0.010)
# inventory.plot(ax=ax, color=inventory["project"], legend=True, markersize=0.010)

Riyadhboundarygdf.to_crs(epsg=3857).plot(edgecolor="black", facecolor="none", ax=ax)

# add a basemap and remove axis
source = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
ctx.add_basemap(ax, source=source)
ax.axis('off')

# Calculate values counts and sort them by order Desc
counts = inventory['project'].value_counts()

# Customize legend labels with the values counts and create handles
sorted_labels = sorted(counts.keys(), key=lambda label: counts[label], reverse=True)
labels = [f"{label} ({counts[label]})" for label in sorted_labels]

index_map = {v: i for i, v in enumerate(sorted_labels)}
color_dict=dict(sorted(color_dict.items(), key=lambda pair: index_map[pair[0]]))
handles = [plt.Rectangle((0, 0), 1, 1, color=color) for label, color in color_dict.items()]
# Manually add 'KKR' at the beginning of sorted_labels list


# Add legend title and plot it
legend_title = "LEGEND \n"
plt.legend(handles, labels, title=legend_title, bbox_to_anchor=(1.25, 1))

# Add a map title and specify size
plt.title("Masterplan Current projects \n KKR", size=18)

# Save the plot to an image(png)
plt.savefig('plot_0.png', dpi='figure', bbox_inches='tight')

plt.show()

2- Masterplan vegetation cover

Input:

          Fulcrum ([TMO] combined inventories)

Output:

          Total canopy area

          RCRC: Existing Canopy
          RCRC: To be replanted canopy
          notRCRC:Other Green Cover

          Total vegetation cover
          Existing vegetation cover

page 13

In [ ]:
# as-builts spread
testKKR['spread'] = testKKR.spread.astype(str).astype(float)

testKKR.loc[testKKR['spread'] < 7, 'spread'] = 7
testKKR.loc[testKKR['spread'].isna(), 'spread'] = 7
testKKR.reset_index(inplace=True, drop=True)

finalnodub = testKKR.drop_duplicates(['geometry'])
final_c = finalnodub.copy()
final_c.reset_index(inplace=True, drop=True)

# Create a buffer per point
final_c.loc[final_c.geometry.type == 'Point','geometry'] = final_c.buffer((final_c['spread']/2))

# Create a list of polygons
polygons = []
for feature in final_c.geometry:
    polygons.append(shapely.geometry.shape(feature))
    
# Extract the geometry of each polygon

# Create a list of unions of intersecting polygons
# TODO: cant you just run dissolve? eg: final_c.dissolve()
unions = []
for i in range(len(polygons)):
    for j in range(i + 1, len(polygons)):
        if polygons[i].intersects(polygons[j]):
            unions.append(polygons[i].union(polygons[j]))
        else:
            unions.append(polygons[i])

# Create a list of areas
areas = []
for polygon in polygons:
    areas.append(polygon.area)

# Calculate the summation of areas
total_area = sum(areas)

# Print the total area
if len(unions) > 0:
    print("total area of vegetation cover = ", total_area)
else:
    print("There are no intersecting polygons, but the summation of areas is ", total_area)

#print the output area:
KKR_area = KKRboundarygdf.area

In [ ]:
# Reproject data to Web Mercator projection (EPSG:3857)
testKKR = testKKR.to_crs(epsg=3857)

# Plot the GeoDataFrame with colors and labels.#cmap: Set3, plasma, magma, inferno, viridis, coolwarm
fig, ax = plt.subplots(figsize=(15, 15))
testKKR.plot(ax=ax, color='green', legend=True, markersize=0.05)
#Riyadhboundarygdf.to_crs(epsg=3857).plot(edgecolor="black", facecolor="none", ax=ax)

# Add basemap
source = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
ctx.add_basemap(ax, source=source)
ax.set_axis_off()

# Add legend title and plot it
legend_title = "LEGEND \n"
handles = [plt.Rectangle((0, 0), 1, 1, color='green')]
label = 'Total vegetation cover: ' + str(round(KKR_area[0],2)) + ' m2'# this needs to be dynamic
plt.legend(handles, [label], title=legend_title, bbox_to_anchor=(1, 1))

# add a title
plt.title('Master Plan Vegetation Cover \n KKR', size=18)

    # Save the plot to a PNG file
plt.savefig('plot_1', dpi='figure', bbox_inches='tight')

    # Show the plot
plt.show()

3- Masterplan Tree Species:

Input:

            Fulcrum ([TMO] combined inventories)
Output:

            Species
      
page 15

In [ ]:
#Creating a dict of colors for eaach value in genus column

color_dict = {'Acacia': 'orange', 'Phoenix': 'purple', 'Ziziphus': 'blue', 'Ficus':'pink', 'Olea': 'olive', 'Prosopis':'brown',
              'No name':'green', 'Moringa':'cyan','Parkinsonia':'green', 'Washingtonia':'green', 'Millettia':'green', 'Albizia': 'green',
              'Pithecellobium': 'green', 'Ceratonia':'green', 'Maerua':'green', 'Dafla':'green', 'Alovera':'green', 'Azadirachta':'green'}
# todo: this needs to be dynamic as the data will always vary between projects and different points in time. 

# Map the color_dict values to the testKKRKSR["genus"] to be correctly mapped on the map
testKKR["color"] = testKKR['genus'].map(color_dict)

#plot data based on the specified testKKRKSR['genus']
fig, ax = plt.subplots(figsize=(20, 20))
testKKR.plot(ax=ax, color=testKKR["color"], legend=True, markersize=0.1)
#Riyadhboundarygdf.to_crs(epsg=3857).plot(edgecolor="black", facecolor="none", ax=ax)

#add a basemap and remove axis
source = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
ctx.add_basemap(ax, source=source)
ax.axis('off')

#Calculate values counts and sort them by order Desc
counts = testKKR['genus'].value_counts()
sorted_labels = sorted(color_dict.keys(), key=lambda label: counts[label], reverse=True)  # Sort by counts in descending order

#customize legend labels with counts and create handles/circle/line and dot...etc.
labels = [f"{label} ({counts[label]})" for label in sorted_labels]
handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in color_dict.values()]

#add legend title and plot it
legend_title = "LEGEND \n"
plt.legend(handles, labels, title=legend_title, bbox_to_anchor=(1, 1))
#legend_text = f"Total Trees: {testKKR.shape[0]}\n" + "\n".join(labels)

# add a map title
plt.title("Masterplan Tree Genus \n KKR", size=18)

#Save the plot to an image(png)
plt.savefig('plot_2.png',  dpi='figure', bbox_inches='tight')

plt.show()

4- Tree Maintenance Status Map

Input:

            Fulcrum ([TMO] combined inventories)
Output:

            Status
            unsuitable trees
            total trees

page 16

In [ ]:
testKKR['status'].unique()

In [ ]:
#Create a dict of colors for eaach value in status column
color_dict = {'No action required': 'limegreen', 'Request of activation': 'orangered','Issue in place':'red'}
# todo: this needs to be dynamic as the data will always vary between projects and different points in time. 

#Map the color_dict values to the testKKRKSR["status"] to be correctly mapped on the map
testKKR["color"] = testKKR["status"].map(color_dict)

#plot data based on the specified testKKRKSR['status']
fig, ax = plt.subplots(figsize=(15, 15))
testKKR.plot(ax=ax, color=testKKR["color"], legend=True, markersize=0.1)
#Riyadhboundarygdf.to_crs(epsg=3857).plot(edgecolor="black", facecolor="none", ax=ax)

#add a basemap and remove axis
source = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
ctx.add_basemap(ax, source=source)
ax.axis('off')

#Calculate values counts and sort them by order Desc
counts = testKKR['status'].value_counts()
sorted_labels = sorted(color_dict.keys(), key=lambda label: counts[label], reverse=True)  # Sort by counts in descending order

# Customize legend labels with counts
labels = [f"{label} ({counts[label]})" for label in sorted_labels]
handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in color_dict.values()]

#add legend title and plot it
legend_title = "LEGEND \n"
plt.legend(handles, labels, title=legend_title, bbox_to_anchor=(1, 1))
#legend_text = f"Total Trees: {testKKR.shape[0]}\n" + "\n".join(labels)

# add a map title
plt.title("Masterplan Current Status \n KKR", size=18)

#Save the plot to an image(png)
plt.savefig('plot_3.png',  dpi='figure', bbox_inches='tight')

plt.show()

Extra - 7 - Masterplan Tree CLasses



Input:

            Fulcrum data ([TMO] combined tree inventory)

Output:

            Palm trees

            Shade trees

In [ ]:
#Create a dict of colors for eaach value in ClASS column
color_dict = {'Shade': 'turquoise', 'Palm': 'deeppink','Not identified':'grey'}

#Map the color_dict values to the testKKRKSR["Class"] to be correctly mapped on the map
testKKR["color"] = testKKR["Class"].map(color_dict)
# todo: this needs to be dynamic as the data will always vary between projects and different points in time. 

#plot data based on the specified testKKRKSR['status']
fig, ax = plt.subplots(figsize=(15, 15))
testKKR.plot(ax=ax, color=testKKR["color"], legend=True, markersize=0.1)
#Riyadhboundarygdf.to_crs(epsg=3857).plot(edgecolor="black", facecolor="none", ax=ax)

#add a basemap and remove axis
source = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
ctx.add_basemap(ax, source=source)
ax.axis('off')

#Calculate values counts and sort them by order Desc
counts = testKKR['Class'].value_counts()
sorted_labels = sorted(color_dict.keys(), key=lambda label: counts[label], reverse=True)  # Sort by counts in descending order

# Customize legend labels with counts and create handles
labels = [f"{label} ({counts[label]})" for label in sorted_labels]
handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in color_dict.values()]

#add legend title and plot it
legend_title = "LEGEND \n"
legend_text = f"Total Trees: {testKKR.shape[0]}\n" + "\n".join(labels)
plt.legend(handles, labels, title=legend_title, bbox_to_anchor=(1, 1))

# add a mp title
plt.title("Masterplan Current Class \n KKR", size=18)

#Save the plot to an image(png)
plt.savefig('plot_4.png',  dpi='figure', bbox_inches='tight')

plt.show()


5- Tree Health Map: health status for Fulcrum less than 6 months, older than 6 much we used NDVI analysis.

Input:

            Fulcrum + ML data
Output:

            Good
            Moderate
            Poor
            Existing trees total
            Dead am missing

page 17
# todo: can you add the final values to the markdown element? 

In [ ]:
spread = testKKR.copy()
# spread.geometry = spread.buffer((spread['spread']/2)) # spread to draw round canopies
spread.reset_index(inplace=True, drop=True)

# make GEE feature collection from buffered KKSRboundary
KKRboxdf32638 = gpd.GeoDataFrame(geometry=pd.DataFrame(KKRboundarygdf.envelope).values.flatten(), crs=32638)
KKRboxdf32639 = KKRboxdf32638.dissolve().envelope
KKRjson32639 = KKRboxdf32639.to_json()
KKRbbox32639 = ee.FeatureCollection(json.loads(KKRjson32639))

######################## make GEE feature collection from buffered KKSRboundary bounding box
KKRboxdf = gpd.GeoDataFrame(geometry=pd.DataFrame(KKRboundarygdf.envelope).values.flatten(), crs=32638)
KKRboxdft = KKRboxdf.dissolve().envelope
KKRjsondft = KKRboxdft.to_crs(4326).to_json()
KKRbbox4326 = ee.FeatureCollection(json.loads(KKRjsondft))

######################## set time frame
today = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")  # number of days as a delimiter
before = (datetime.today() - timedelta(days=20)).strftime("%Y-%m-%d")  # number of days as a delimiter
#days_in_interval = today - before
days_in_interval = 20

# get Sentinel data
collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
              .filterDate(before, today)
              .filterBounds(KKRbbox4326.geometry())
              .map(lambda image: image.clip(KKRbbox4326.geometry()))
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
              .map(lambda image: image.normalizedDifference(['B8', 'B4']).rename('ndvi'))
              )
x1 = str(KKRbbox32639.geometry().getInfo()['coordinates'][0][0]).strip("[]").split(',')[0]
y1 = str(KKRbbox32639.geometry().getInfo()['coordinates'][0][1]).strip("[]").split(',')[1]
transform = from_origin(float(x1), float(y1), 10, 10)

################################### date check
a=0
imgList = collection.toList(100)
for imga in imgList.getInfo():
	a=a+1
	print(a, imga['properties']['system:index'])

##################################### get first image
def update_code(collection):
    band_names = collection.getInfo()['bands']
    new_collection = ee.ImageCollection(
        collection.map(lambda image: image.select(band_names).rename('value'))
    )
    return new_collection

collection = update_code(collection)

imgfirst = collection.limit(1, 'system:index').first()
arrfirst = imgfirst.sampleRectangle(region = KKRbbox4326.geometry())
arrndvifirst = arrfirst.get('ndvi')
nparrndvifirst = np.array(arrndvifirst.getInfo())
nparrndvifirstt = nparrndvifirst.astype(np.float32)

new_dataset = rasterio.open('firstimg.tif', 'w', driver='GTiff',
                                              height=nparrndvifirstt.size, width=nparrndvifirstt.size,
                                              count=1, dtype=str(nparrndvifirstt.dtype), nodata=-99999999,
                                              crs=32638)
new_dataset.write(np.array([[1,nparrndvifirstt]]), 1)
new_dataset.close()

########### process first image
with rasterio.open("firstimg.tif") as src:
	affine = src.transform
	ndval = src.nodatavals[0]
	array = src.read(1)
	array = array.astype('float64')
	array[array==ndval] = np.nan
	df_zonal_stats = pd.DataFrame(zonal_stats(spread, array, stats='max', affine=affine, nodata=-99999999, all_touched=True))

# adding statistics back to original GeoDataFrame
zsresults = pd.concat([spread, df_zonal_stats], axis=1)

############# NDVI health (new categories due to very negative results, violoating the 0.2 threshold)
# todo: we need to check the logic here. Usually the NDVI threshold is 0.2
zsresults.loc[(zsresults['max']>=0.5) ,'ndvi_health_cat'] = 'good - جيد'
zsresults.loc[(zsresults['max']>=0.3) & (zsresults['max'] < 0.5) ,'ndvi_health_cat'] = 'fair - معتدل'
zsresults.loc[(zsresults['max']>0.1) & (zsresults['max'] < 0.3) ,'ndvi_health_cat'] = 'poor - سئ'
zsresults.loc[(zsresults['max']<=0.1) ,'ndvi_health_cat'] = 'dead - ميت'
zsresults.rename(columns = {'ndvi_health_cat' : 'ndvi_health_cat'+imgfirst.getInfo()['properties']['system:index'].split("T")[0]}, inplace = True)

zsresults.loc[(zsresults['max'] >= 0.2) ,'ndvi_health'] = 1
zsresults.loc[(zsresults['max'] < 0.2) ,'ndvi_health'] = 0

zsresults.rename(columns = {'ndvi_health': 'ndvi_health_' + imgfirst.getInfo()['properties']['system:index'].split("T")[0]}, inplace = True)
zsresults.rename(columns = {'max':'max_' + imgfirst.getInfo()['properties']['system:index'].split("T")[0]}, inplace = True)

# Process the last image
lastimg = collection.limit(1, 'system:index', False).first()
arrlast = lastimg.sampleRectangle(region=KKRbbox4326.geometry())
arrndvilast = arrlast.get('ndvi')
nparrndvilast = np.array(arrndvilast.getInfo())
nparrndvilastt = nparrndvilast.astype(np.float32)

# Create raster file for the last image
new_dataset_last = rasterio.open('lastimg.tif', 'w', driver='GTiff',
                                 height=nparrndvilastt.size, width=nparrndvilastt.size,
                                 count=1, dtype=nparrndvilastt.dtype, nodata=-99999999,
                                 crs=32638)
new_dataset_last.write(np.array([[1,nparrndvilastt]]), 1)
new_dataset_last.close()


############# process last image
with rasterio.open("lastimg.tif") as src:
	affine = src.transform
	ndval = src.nodatavals[0]
	array = src.read(1)
	array = array.astype('float64')
	array[array==ndval] = np.nan
	df_zonal_stats = pd.DataFrame(zonal_stats(spread, array, stats='max', affine=affine, nodata=-99999999, all_touched=True))

# adding statistics back to original GeoDataFrame
zsresults = pd.concat([zsresults, df_zonal_stats], axis=1)

################ NDVI health (new categories due to very negative results, violoating the 0.2 threshold)
zsresults.loc[(zsresults['max']>=0.5) ,'ndvi_health_cat'] = 'good - جيد'
zsresults.loc[(zsresults['max']>=0.3) & (zsresults['max'] < 0.5) ,'ndvi_health_cat'] = 'fair - معتدل'
zsresults.loc[(zsresults['max']>0.1) & (zsresults['max'] < 0.3) ,'ndvi_health_cat'] = 'poor - سئ'
zsresults.loc[(zsresults['max']<=0.1) ,'ndvi_health_cat'] = 'dead - ميت'

zsresults.rename(columns = {'ndvi_health_cat' : 'ndvi_health_cat'+lastimg.getInfo()['properties']['system:index'].split("T")[0]}, inplace = True)

zsresults.loc[(zsresults['max'] >= 0.2), 'ndvi_health'] = 1
zsresults.loc[(zsresults['max'] < 0.2), 'ndvi_health'] = 0

zsresults.rename(columns = {'ndvi_health' : 'ndvi_health_'+lastimg.getInfo()['properties']['system:index'].split("T")[0]}, inplace = True)
zsresults.rename(columns = {'max' : 'max_' + lastimg.getInfo()['properties']['system:index'].split("T")[0]}, inplace = True)

########################################## diff
zsresults['diff'] = zsresults['ndvi_health_'+lastimg.getInfo()['properties']['system:index'].split("T")[0]]-zsresults['ndvi_health_'+imgfirst.getInfo()['properties']['system:index'].split("T")[0]]
#print(zsresults['diff'].value_counts())

# print(lastimg.getInfo()['properties']['system:index'].split("T")[0])
# print(imgfirst.getInfo()['properties']['system:index'].split("T")[0])
# print(zsresults['ndvi_health_'+lastimg.getInfo()['properties']['system:index'].split("T")[0]])
# print(zsresults['ndvi_health_cat'+lastimg.getInfo()['properties']['system:index'].split("T")[0]].value_counts())


# if health older than 6 month use NDVI health
zsresults['health_merge'] = zsresults['health']
zsresults.loc[(zsresults['health_merge']=='NaN'), 'health_merge'] =  zsresults['ndvi_health_cat' + lastimg.getInfo()['properties']['system:index'].split("T")[0]]
zsresults.loc[(zsresults['health_merge'].isna()), 'health_merge'] =  zsresults['ndvi_health_cat' + lastimg.getInfo()['properties']['system:index'].split("T")[0]]

####print
print(zsresults.value_counts(['health_merge']))
print(zsresults.isna().value_counts(['health_merge']))

###save to file
#zsresults.to_file(r"/content/drive/MyDrive/BPLA/20230510-DQ-inv-ml-ndvi-health-1.gpkg", driver='GPKG', layer='DQ-inv-ml-ndvi-volume-corrected-74-2')


In [ ]:
for i in range(len(zsresults)):
    if zsresults.loc[i, 'health_merge'] == 'good - جيد':
        zsresults.loc[i, 'health_updated'] = 'Good'
    elif zsresults.loc[i, 'health_merge'] == 'fair - معتدل':
        zsresults.loc[i, 'health_updated'] = 'Moderate'
    elif zsresults.loc[i, 'health_merge'] == 'poor - سئ':
        zsresults.loc[i, 'health_updated'] = 'Poor'
    elif zsresults.loc[i, 'health_merge'] == 'excellent - ممتاز':
        zsresults.loc[i, 'health_updated'] = 'Excellent'
    elif zsresults.loc[i, 'health_merge'] == 'dead - ميت':
        zsresults.loc[i, 'health_updated'] = 'Dead'
    else:
        zsresults.loc[i, 'health_updated'] = 'Missing'


In [ ]:
zsresults = zsresults.to_crs(epsg=3857)

#Create a dict of colors for eaach value in health_updated column
color_dict = {'Missing': 'pink', 'Good': 'lime', 'Moderate': 'orange', 'Poor':'red', 'Excellent':'blue', 'Dead':'black' }

#Map the color_dict values to the testKKRKSR["Class"] to be correctly mapped on the map
zsresults["color"] = zsresults["health_updated"].map(color_dict)

#plot data based on the specified testKKRKSR['status']
fig, ax = plt.subplots(figsize=(15, 15))
zsresults.plot(ax=ax, color=zsresults["color"], legend=True, markersize=0.1)
KKRboundarygdf.to_crs(epsg=3857).plot(edgecolor="black", facecolor="none", ax=ax)

#add a basemap and remove axis
source = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
ctx.add_basemap(ax, source=source)
ax.axis('off')

#Calculate values counts and sort them by order Desc
counts = zsresults['health_updated'].value_counts()
sorted_labels = sorted(color_dict.keys(), key=lambda label: counts[label], reverse=True)  # Sort by counts in descending order

# Customize legend labels with counts and create handles
labels = [f"{label} ({counts[label]})" for label in sorted_labels]
handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in color_dict.values()]

#add legend title and plot it
legend_title = "LEGEND \n"
legend_text = f"Total Trees: {zsresults.shape[0]}\n" + "\n".join(labels)
plt.legend(handles, labels, title=legend_title, bbox_to_anchor=(1, 1))

# add a mp title
plt.title("Masterplan NDVI Health map \n KKR", size=18)

#Save the plot to an image(png)
plt.savefig('plot_6.png',  dpi='figure', bbox_inches='tight')

plt.show()


6- Tree Canopy Change Alert Map

Input:

           Fulcrum + ML data
Output:

          Total number of trees
          Total number of TBR
          Total number of existing trees

          No change detected
          Expansion detected
          Deterioration Detected

page 18

In [ ]:
for i in range(len(zsresults)):
    if zsresults.iloc[i]['diff'] == 0:
        zsresults.loc[i, 'comparisondiff'] = 'No change'
    elif zsresults.iloc[i]['diff'] > 0:
        zsresults.loc[i, 'comparisondiff'] = 'Growth'
    else:
        zsresults.loc[i, 'comparisondiff'] = 'Deterioration'


#Save the plot to an image(png)
plt.savefig('plot_9.png', dpi=300)

#Plot the map
plot_map(zsresults, KKRboundarygdf, 'comparisondiff','Tree NDVI Change Map','plot_10.png')


# Draw them in a pdf report

In [ ]:
w, h = A4
pdf = canvas.Canvas('plots.pdf', pagesize=A4) #letter

for i in range(5):
  image_path = f'plot_{i}.png'
  pdf.drawImage(f'plot_{i}.png', 10, h-500, width=550, height=500)
  pdf.showPage()

pdf.save()
# todo: this works but the plots are distorted
# todo: add a scalebar and north arrow
# todo: do the colour match jessicas HEX codes?
